In [ ]:
import numpy as np
import os
import numpy as np
import uncertainties as u
from uncertainties import ufloat
from uncertainties.umath import sqrt
from uncertainties.umath import __all__
from uncertainties.umath import *
import uncertainties.unumpy as unp
import scipy.constants
import os
import matplotlib.pyplot as plt
from scipy.stats import linregress
import pandas # pip install Jinja2
import pint # pip install pint OR conda install pint
import warnings

warnings.simplefilter("ignore")

units = pint.UnitRegistry(autoconvert_offset_to_baseunit = True)
units.default_format = '~'
units.setup_matplotlib(True)

ulog = u.wrap(np.log)
upower = u.wrap(np.power)
umultiply = u.wrap(np.multiply)
udivide = u.wrap(np.divide)
umean = u.wrap(np.mean)

cm = 1/2.54 # For matplotlib figure sizes

In [ ]:
# Area of a Circle
def CircleArea(radius):
    return np.pi * (radius**2)
uCircleArea = u.wrap(CircleArea)

# Goal: Measure $\varphi$

## Given
$l_{2} = (70.0\pm0.2)~\mathrm{mm}, R_{2} = (10.20\pm0.01)~\mathrm{m\Omega}, D = (5.00\pm0.02)~\mathrm{mm}, S=(19.63\pm0.16)~\mathrm{mm^2}, A_{Nickel-Kupfer} = (20.40\pm0.05)~\mathrm{\mu V~K^{-1}}$

In [ ]:
u_l2 = u.ufloat(70.0, 0.2) * units.mm
u_R2 = u.ufloat(10.2, 0.01) * units.milliohm
u_D = u.ufloat(5.00, 0.02) * units.mm
u_S = uCircleArea(u_D/2.0)
u_A__nicker_kupfer = u.ufloat(20.4, 0.05) * units.microvolt / units.degree_Kelvin

## Measure $t_{Celsius}$

In [ ]:
# TODO MEASURE t IN CELSIUS
u_t = u.ufloat(99.9,99.9) * units.degreeC
u_T = u_t.to(units.degree_Kelvin)

## For four Current values $I_{1}$ between 0.25 and 1 Ampere measure $I_{2}$

In [ ]:
# TODO WHAT IS THE UNCERTAINTY OF I MEASUREMENT
I_unc = 99.99

# TODO
I_meas_unit = units.ampere # WHAT UNIT DO WE MEASURE I IN?
# TODO
I1_meas = [0.25, 0.5, 0.75, 1.0] # WHAT VALUES OF I1 WERE USED?
# TODO
I2_meas = [99.9, 99.9, 99.9, 99.9] # WHAT CORRESPONDING VALUES OF I2 WERE MEASURED?

u_I1 = np.array([u.ufloat(I, I_unc)*I_meas_unit for I in I1_meas]) # I1
u_I2 = np.array([u.ufloat(I, I_unc)*I_meas_unit for I in I2_meas]) # I2

u_I1_divby_I2 = udivide(u_I1, u_I2) # I1 / I2
u_Mean_I1divI2 = umean(u_I1_divby_I2) # <(I1/I2)>

## Calculate the electric conductivity $\sigma = \langle (\frac{I_{1}}{I_{2}}) \rangle \frac{l_{2}}{S\cdot R_{2}}$

In [ ]:
def CalculateSigma(u_Mean_I1byI2, u_SurfaceArea, u_length, u_Resistance):
    return u_Mean_I1divI2 * u_length / (u_SurfaceArea*u_Resistance)
uCalculateSigma = u.wrap(CalculateSigma)

In [ ]:
u_Sigma = uCalculateSigma(u_Mean_I1divI2, u_S, u_l2, u_R2).to_base_units().to_compact()
u_Sigma

### Generate a Table of ($I_{1}$, $I_{2}$, $\frac{I_{1}}{I_{2}}$)

In [50]:
print("TABLE (I1, I2, I1/I2):")
df_I1_I2_I1byN2 = pandas.DataFrame({'I1': u_I1, 'I2': u_I2, 'I1/I2': u_I1_divby_I2})
df_I1_I2_I1byN2.transpose()
print(df_I1_I2_I1byN2.to_latex(index=False))

TABLE (I1, I2, I1/I2):
\begin{tabular}{lll}
\toprule
               I1 &                I2 &     I1/I2 \\
\midrule
(1.0+/-1.0)e+02 A & (1.0+/-1.0)e+02 A & 1.0+/-1.4 \\
(1.0+/-1.0)e+02 A & (1.0+/-1.0)e+02 A & 1.0+/-1.4 \\
(1.0+/-1.0)e+02 A & (1.0+/-1.0)e+02 A & 1.0+/-1.4 \\
(1.0+/-1.0)e+02 A & (1.0+/-1.0)e+02 A & 1.0+/-1.4 \\
\bottomrule
\end{tabular}



In [46]:
print('Mean (I1/I2)', u_Mean_I1divI2)

Mean (I1/I2) 1.0+/-0.7


In [54]:
print('Sigma = {:.2u~P}'.format(u_Sigma))
print('\n\n\n')
print('Sigma = {:.2u~L}'.format(u_Sigma))

Sigma = (3.5+/-2.5)e+05 A²·s³/kg/m³




Sigma = (3.5+/-2.5)e+05\ \frac{\mathrm{A}^{2} \cdot \mathrm{s}^{3}}{\left(\mathrm{kg} \cdot \mathrm{m}^{3}\right)}


# DONE

In [55]:
# For task 3
print(u_T)

(3.7+/-1.0)e+02 K
